In [ ]:
import urllib.request
import shutil
import numpy as np
import matplotlib.pyplot as plt
import os
import glob


import numpy as np
import pickle
import cv2
import math
import os
import random
import tensorflow as tf

In [ ]:
def make_video(images, outvid=None, fps=15, size=None,
               is_color=True, format='MP42'):
    """
    Create a video from a list of images.
 
    @param      outvid      output video
    @param      images      list of images to use in the video
    @param      fps         frame per second
    @param      size        size of each frame
    @param      is_color    color
    @param      format      see http://www.fourcc.org/codecs.php
    @return                 see http://opencv-python-tutroals.readthedocs.org/en/latest/py_tutorials/py_gui/py_video_display/py_video_display.html
 
    The function relies on http://opencv-python-tutroals.readthedocs.org/en/latest/.
    By default, the video will have the size of the first image.
    It will resize every image to this size before adding them to the video.
    MODIFIED FROM: http://www.xavierdupre.fr/blog/2016-03-30_nojs.html
    """
    from cv2 import VideoWriter, VideoWriter_fourcc, imread, resize
    fourcc = VideoWriter_fourcc(*format)
    vid = None
    for image in images:
        #print(image)
        if not os.path.exists(image):
            raise FileNotFoundError(image)
        img = imread(image)
        if vid is None:
            if size is None:
                size = img.shape[1], img.shape[0]
            vid = VideoWriter(outvid, fourcc, float(fps), size, is_color)
        if size[0] != img.shape[1] and size[1] != img.shape[0]:
            img = resize(img, size)
        vid.write(img)
    vid.release()
    return vid


def multiple_sort(value):
    """sorts zt_data, bt_data, ht_data lists alphabetically and numerically """
    values = value[0].split("/")
    key1 = values[2]
    key2 = values[4]
    key3 = values[3]
    key4 = int(values[5].split('.')[0])
    return key1, key2, key3, key4

In [ ]:
# Copyright (c) 2020, NVIDIA Corporation. All rights reserved.
#
# This work is made available
# under the Nvidia Source Code License (1-way Commercial).
# To view a copy of this license, visit
# https://nvlabs.github.io/Dancing2Music/License.txt
import os  
import pickle
import numpy as np
import random
import torch.utils.data
from torchvision.datasets import ImageFolder
#import utils


class PoseDataset(torch.utils.data.Dataset):
  def __init__(self, data_dir, tolerance=False):
    self.data_dir = data_dir
    z_fname = '{}/unitList/zumba_unit.txt'.format(data_dir)
    b_fname = '{}/unitList/ballet_unit.txt'.format(data_dir)
    h_fname = '{}/unitList/hiphop_unit.txt'.format(data_dir)
    self.z_data = []
    self.b_data = []
    self.h_data = []
    with open(z_fname, 'r') as f:
      for line in f:
        self.z_data.append([s for s in line.strip().split(' ')])
    with open(b_fname, 'r') as f:
      for line in f:
        self.b_data.append([s for s in line.strip().split(' ')])
    with open(h_fname, 'r') as f:
      for line in f:
        self.h_data.append([s for s in line.strip().split(' ')])
    self.data = [self.z_data, self.b_data, self.h_data]

    self.tolerance = tolerance
    if self.tolerance:
      z3_fname = '{}/unitList/zumba_unitseq3.txt'.format(data_dir)
      b3_fname = '{}/unitList/ballet_unitseq3.txt'.format(data_dir)
      h3_fname = '{}/unitList/hiphop_unitseq3.txt'.format(data_dir)
      z4_fname = '{}/unitList/zumba_unitseq4.txt'.format(data_dir)
      b4_fname = '{}/unitList/ballet_unitseq4.txt'.format(data_dir)
      h4_fname = '{}/unitList/hiphop_unitseq4.txt'.format(data_dir)
      z3_data = []; b3_data = []; h3_data = []; z4_data = []; b4_data = []; h4_data = []
      with open(z3_fname, 'r') as f:
        for line in f:
          z3_data.append([s for s in line.strip().split(' ')])
      with open(b3_fname, 'r') as f:
        for line in f:
          b3_data.append([s for s in line.strip().split(' ')])
      with open(h3_fname, 'r') as f:
        for line in f:
          h3_data.append([s for s in line.strip().split(' ')])
      with open(z4_fname, 'r') as f:
        for line in f:
          z4_data.append([s for s in line.strip().split(' ')])
      with open(b4_fname, 'r') as f:
        for line in f:
          b4_data.append([s for s in line.strip().split(' ')])
      with open(h4_fname, 'r') as f:
        for line in f:
          h4_data.append([s for s in line.strip().split(' ')])
      self.zt_data = z3_data + z4_data
      self.bt_data = b3_data + b4_data
      self.ht_data = h3_data + h4_data
      self.t_data = [self.zt_data, self.bt_data, self.ht_data]

    self.mean_pose=np.load(data_dir+'/stats/all_onbeat_mean.npy')
    self.std_pose=np.load(data_dir+'/stats/all_onbeat_std.npy')

  def __getitem__(self, index):
    cls = random.randint(0,2)
    cls = random.randint(0,1)
    if self.tolerance and random.randint(0,9)==0:
      index = random.randint(0, len(self.t_data[cls])-1)
      path = self.t_data[cls][index][0]
      path = os.path.join(self.data_dir, path[5:])
      orig_poses = np.load(path)
      sel = random.randint(0, orig_poses.shape[0]-1)
      orig_poses = orig_poses[sel]
    else:
      index = random.randint(0, len(self.data[cls])-1)
      path = self.data[cls][index][0]
      path = os.path.join(self.data_dir, path[5:])
      orig_poses = np.load(path)

    xjit = np.random.uniform(low=-50, high=50)
    yjit = np.random.uniform(low=-20, high=20)
    poses = orig_poses.copy()
    poses[:,:,0] += xjit
    poses[:,:,1] += yjit
    xjit = np.random.uniform(low=-50, high=50)
    yjit = np.random.uniform(low=-20, high=20)
    poses2 = orig_poses.copy()
    poses2[:,:,0] += xjit
    poses2[:,:,1] += yjit

    poses = poses.reshape(poses.shape[0], poses.shape[1]*poses.shape[2])
    poses2 = poses2.reshape(poses2.shape[0], poses2.shape[1]*poses2.shape[2])
    for i in range(poses.shape[0]):
      poses[i] = (poses[i]-self.mean_pose)/self.std_pose
      poses2[i] = (poses2[i]-self.mean_pose)/self.std_pose

    return torch.Tensor(poses), torch.Tensor(poses2)

  def __len__(self):
    return len(self.z_data)+len(self.b_data)


class MovementAudDataset(torch.utils.data.Dataset):
  def __init__(self, data_dir):
    self.data_dir = data_dir
    z3_fname = '{}/unitList/zumba_unitseq3.txt'.format(data_dir)
    b3_fname = '{}/unitList/ballet_unitseq3.txt'.format(data_dir)
    h3_fname = '{}/unitList/hiphop_unitseq3.txt'.format(data_dir)
    z4_fname = '{}/unitList/zumba_unitseq4.txt'.format(data_dir)
    b4_fname = '{}/unitList/ballet_unitseq4.txt'.format(data_dir)
    h4_fname = '{}/unitList/hiphop_unitseq4.txt'.format(data_dir)
    self.z3_data = []
    self.b3_data = []
    self.h3_data = []
    self.z4_data = []
    self.b4_data = []
    self.h4_data = []
    with open(z3_fname, 'r') as f:
      for line in f:
        self.z3_data.append([s for s in line.strip().split(' ')])
    with open(b3_fname, 'r') as f:
      for line in f:
        self.b3_data.append([s for s in line.strip().split(' ')])
    with open(h3_fname, 'r') as f:
      for line in f:
        self.h3_data.append([s for s in line.strip().split(' ')])
    with open(z4_fname, 'r') as f:
      for line in f:
        self.z4_data.append([s for s in line.strip().split(' ')])
    with open(b4_fname, 'r') as f:
      for line in f:
        self.b4_data.append([s for s in line.strip().split(' ')])
    with open(h4_fname, 'r') as f:
      for line in f:
        self.h4_data.append([s for s in line.strip().split(' ')])
    self.data_3 = [self.z3_data, self.b3_data, self.h3_data]
    self.data_4 = [self.z4_data, self.b4_data, self.h4_data]

    z_data_root = 'zumba/'
    b_data_root = 'ballet/'
    h_data_root = 'hiphop/'
    self.data_root = [z_data_root, b_data_root, h_data_root ]
    self.mean_pose=np.load(data_dir+'/stats/all_onbeat_mean.npy')
    self.std_pose=np.load(data_dir+'/stats/all_onbeat_std.npy')
    self.mean_aud=np.load(data_dir+'/stats/all_aud_mean.npy')
    self.std_aud=np.load(data_dir+'/stats/all_aud_std.npy')

  def __getitem__(self, index):
    cls = random.randint(0,2)
    cls = random.randint(0,1)
    isthree = random.randint(0,1)

    if isthree == 0:
      index = random.randint(0, len(self.data_4[cls])-1)
      path = self.data_4[cls][index][0]
    else:
      index = random.randint(0, len(self.data_3[cls])-1)
      path = self.data_3[cls][index][0]
    path = os.path.join(self.data_dir, path[5:])
    stdpSeq = np.load(path)
    vid, cid = path.split('/')[-4], path.split('/')[-3]
    #vid, cid = vid_cid[:11], vid_cid[12:]
    aud = np.load('{}/{}/{}/{}/aud/c{}_fps15.npy'.format(self.data_dir, self.data_root[cls], vid, cid, cid))

    stdpSeq = stdpSeq.reshape(stdpSeq.shape[0], stdpSeq.shape[1], stdpSeq.shape[2]*stdpSeq.shape[3])
    for i in range(stdpSeq.shape[0]):
      for j in range(stdpSeq.shape[1]):
        stdpSeq[i,j] = (stdpSeq[i,j]-self.mean_pose)/self.std_pose
    if isthree == 0:
      start = random.randint(0,1)
      stdpSeq = stdpSeq[start:start+3]

    for i in range(aud.shape[0]):
      aud[i] = (aud[i]-self.mean_aud)/self.std_aud
    aud = aud[:30]
    return torch.Tensor(stdpSeq), torch.Tensor(aud)

  def __len__(self):
    return len(self.z3_data)+len(self.b3_data)+len(self.z4_data)+len(self.b4_data)+len(self.h3_data)+len(self.h4_data)

def get_loader(batch_size, shuffle, num_workers, dataset, data_dir, tolerance=False):
  if dataset == 0:
    a2d = PoseDataset(data_dir, tolerance)
  elif dataset == 2:
    a2d = MovementAudDataset(data_dir)
  data_loader = torch.utils.data.DataLoader(dataset=a2d,
                                            batch_size=batch_size,
                                            shuffle=shuffle,
                                            num_workers=num_workers,
                                            )
  return data_loader


In [ ]:
# Copyright (c) 2020, NVIDIA Corporation. All rights reserved.
#
# This work is made available
# under the Nvidia Source Code License (1-way Commercial).
# To view a copy of this license, visit
# https://nvlabs.github.io/Dancing2Music/License.txt

import numpy as np
import pickle
import cv2
import math
import os
import random
import tensorflow as tf

class Logger(object):
  def __init__(self, log_dir):
    self.writer = tf.summary.FileWriter(log_dir)

  def scalar_summary(self, tag, value, step):
    summary = tf.Summary(value=[tf.Summary.Value(tag=tag, simple_value=value)])
    self.writer.add_summary(summary, step)

def vis(poses, outdir, aud=None):
  colors = [[255, 0, 0], [255, 85, 0], [255, 170, 0], [255, 255, 0], [170, 255, 0], [85, 255, 0], [0, 255, 0], \
          [0, 255, 85], [0, 255, 170], [0, 255, 255], [0, 170, 255], [0, 85, 255], [0, 0, 255], [85, 0, 255], \
          [170, 0, 255], [255, 0, 255], [255, 0, 170], [255, 0, 85]]

  # find connection in the specified sequence, center 29 is in the position 15
  limbSeq = [[2,3], [2,6], [3,4], [4,5], [6,7], [7,8], [2,9], [9,10], \
           [10,11], [2,12], [12,13], [13,14], [2,1], [1,15], [15,17], \
           [1,16], [16,18], [3,17], [6,18]]

  neglect = [14,15,16,17]

  for t in range(poses.shape[0]):
    #break
    canvas = np.ones((256,500,3), np.uint8)*255

    thisPeak = poses[t]
    for i in range(18):
      if i in neglect:
        continue
      if thisPeak[i,0] == -1:
        continue
      cv2.circle(canvas, tuple(thisPeak[i,0:2].astype(int)), 4, colors[i], thickness=-1)

    for i in range(17):
      limbid = np.array(limbSeq[i])-1
      if limbid[0] in neglect or limbid[1] in neglect:
        continue
      X = thisPeak[[limbid[0],limbid[1]], 1]
      Y = thisPeak[[limbid[0],limbid[1]], 0]
      if X[0] == -1 or Y[0]==-1 or X[1]==-1 or Y[1]==-1:
        continue
      stickwidth = 4
      cur_canvas = canvas.copy()
      mX = np.mean(X)
      mY = np.mean(Y)
      #print(X, Y, limbid)
      length = ((X[0] - X[1]) ** 2 + (Y[0] - Y[1]) ** 2) ** 0.5
      angle = math.degrees(math.atan2(X[0] - X[1], Y[0] - Y[1]))
      polygon = cv2.ellipse2Poly((int(mY),int(mX)), (int(length/2), stickwidth), int(angle), 0, 360, 1)
      #print(i, n, int(mY), int(mX), limbid, X, Y)
      cv2.fillConvexPoly(cur_canvas, polygon, colors[i])
      canvas = cv2.addWeighted(canvas, 0.4, cur_canvas, 0.6, 0)
    if aud is not None:
      if aud[:,t] == 1:
        cv2.circle(canvas, (30, 30), 20, (0,0,255), -1)
        #canvas = cv2.copyMakeBorder(canvas,10,10,10,10,cv2.BORDER_CONSTANT,value=[255,0,0])
    cv2.imwrite(os.path.join(outdir, 'frame{0:03d}.png'.format(t)),canvas)

def vis2(poses, outdir, fibeat):
  colors = [[255, 0, 0], [255, 85, 0], [255, 170, 0], [255, 255, 0], [170, 255, 0], [85, 255, 0], [0, 255, 0], \
          [0, 255, 85], [0, 255, 170], [0, 255, 255], [0, 170, 255], [0, 85, 255], [0, 0, 255], [85, 0, 255], \
          [170, 0, 255], [255, 0, 255], [255, 0, 170], [255, 0, 85]]

  # find connection in the specified sequence, center 29 is in the position 15
  limbSeq = [[2,3], [2,6], [3,4], [4,5], [6,7], [7,8], [2,9], [9,10], \
           [10,11], [2,12], [12,13], [13,14], [2,1], [1,15], [15,17], \
           [1,16], [16,18], [3,17], [6,18]]


  neglect = [14,15,16,17]

  ibeat = cv2.imread(fibeat);
  ibeat = cv2.resize(ibeat, (500,200))

  for t in range(poses.shape[0]):
    subibeat = ibeat.copy()
    canvas = np.ones((256+200,500,3), np.uint8)*255
    canvas[256:,:,:] = subibeat

    overlay = canvas.copy()
    cv2.rectangle(overlay, (int(500/poses.shape[0]*(t+1)),256),(500,256+200), (100,100,100), -1)
    cv2.addWeighted(overlay, 0.4, canvas, 1-0.4, 0, canvas)
    thisPeak = poses[t]
    for i in range(18):
      if i in neglect:
        continue
      if thisPeak[i,0] == -1:
        continue
      cv2.circle(canvas, tuple(thisPeak[i,0:2].astype(int)), 4, colors[i], thickness=-1)

    for i in range(17):
      limbid = np.array(limbSeq[i])-1
      if limbid[0] in neglect or limbid[1] in neglect:
        continue
      X = thisPeak[[limbid[0],limbid[1]], 1]
      Y = thisPeak[[limbid[0],limbid[1]], 0]
      if X[0] == -1 or Y[0]==-1 or X[1]==-1 or Y[1]==-1:
        continue
      stickwidth = 4
      cur_canvas = canvas.copy()
      mX = np.mean(X)
      mY = np.mean(Y)
      #print(X, Y, limbid)
      length = ((X[0] - X[1]) ** 2 + (Y[0] - Y[1]) ** 2) ** 0.5
      angle = math.degrees(math.atan2(X[0] - X[1], Y[0] - Y[1]))
      polygon = cv2.ellipse2Poly((int(mY),int(mX)), (int(length/2), stickwidth), int(angle), 0, 360, 1)
      #print(i, n, int(mY), int(mX), limbid, X, Y)
      cv2.fillConvexPoly(cur_canvas, polygon, colors[i])
      canvas = cv2.addWeighted(canvas, 0.4, cur_canvas, 0.6, 0)
    cv2.imwrite(os.path.join(outdir, 'frame{0:03d}.png'.format(t)),canvas)

def vis_single(pose, outfile):
  colors = [[255, 0, 0], [255, 85, 0], [255, 170, 0], [255, 255, 0], [170, 255, 0], [85, 255, 0], [0, 255, 0], \
          [0, 255, 85], [0, 255, 170], [0, 255, 255], [0, 170, 255], [0, 85, 255], [0, 0, 255], [85, 0, 255], \
          [170, 0, 255], [255, 0, 255], [255, 0, 170], [255, 0, 85]]

  # find connection in the specified sequence, center 29 is in the position 15
  limbSeq = [[2,3], [2,6], [3,4], [4,5], [6,7], [7,8], [2,9], [9,10], \
           [10,11], [2,12], [12,13], [13,14], [2,1], [1,15], [15,17], \
           [1,16], [16,18], [3,17], [6,18]]

  neglect = [14,15,16,17]

  for t in range(1):
    #break
    canvas = np.ones((256,500,3), np.uint8)*255

    thisPeak = pose
    for i in range(18):
      if i in neglect:
        continue
      if thisPeak[i,0] == -1:
        continue
      cv2.circle(canvas, tuple(thisPeak[i,0:2].astype(int)), 4, colors[i], thickness=-1)

    for i in range(17):
      limbid = np.array(limbSeq[i])-1
      if limbid[0] in neglect or limbid[1] in neglect:
        continue
      X = thisPeak[[limbid[0],limbid[1]], 1]
      Y = thisPeak[[limbid[0],limbid[1]], 0]
      if X[0] == -1 or Y[0]==-1 or X[1]==-1 or Y[1]==-1:
        continue
      stickwidth = 4
      cur_canvas = canvas.copy()
      mX = np.mean(X)
      mY = np.mean(Y)
      length = ((X[0] - X[1]) ** 2 + (Y[0] - Y[1]) ** 2) ** 0.5
      angle = math.degrees(math.atan2(X[0] - X[1], Y[0] - Y[1]))
      polygon = cv2.ellipse2Poly((int(mY),int(mX)), (int(length/2), stickwidth), int(angle), 0, 360, 1)
      cv2.fillConvexPoly(cur_canvas, polygon, colors[i])
      canvas = cv2.addWeighted(canvas, 0.4, cur_canvas, 0.6, 0)
    cv2.imwrite(outfile,canvas)


In [ ]:
data_dir = "/data/Dancing2Music/data/"
tolerance = True
if tolerance:
  z3_fname = '{}/unitList/zumba_unitseq3.txt'.format(data_dir)
  b3_fname = '{}/unitList/ballet_unitseq3.txt'.format(data_dir)
  h3_fname = '{}/unitList/hiphop_unitseq3.txt'.format(data_dir)
  z4_fname = '{}/unitList/zumba_unitseq4.txt'.format(data_dir)
  b4_fname = '{}/unitList/ballet_unitseq4.txt'.format(data_dir)
  h4_fname = '{}/unitList/hiphop_unitseq4.txt'.format(data_dir)
  z3_data = []; b3_data = []; h3_data = []; z4_data = []; b4_data = []; h4_data = []
  with open(z3_fname, 'r') as f:
    for line in f:
      z3_data.append([s for s in line.strip().split(' ')])
  with open(b3_fname, 'r') as f:
    for line in f:
      b3_data.append([s for s in line.strip().split(' ')])
  with open(h3_fname, 'r') as f:
    for line in f:
      h3_data.append([s for s in line.strip().split(' ')])
  with open(z4_fname, 'r') as f:
    for line in f:
      z4_data.append([s for s in line.strip().split(' ')])
  with open(b4_fname, 'r') as f:
    for line in f:
      b4_data.append([s for s in line.strip().split(' ')])
  with open(h4_fname, 'r') as f:
    for line in f:
      h4_data.append([s for s in line.strip().split(' ')])
  zt_data = z3_data + z4_data
  bt_data = b3_data + b4_data
  ht_data = h3_data + h4_data
  t_data = [zt_data, bt_data, ht_data]